In [51]:
import cartolafc
import pandas as pd
from difflib import get_close_matches
import json

pd.set_option('display.max_columns', 50)            # permite a visualização de 50 colunas do dataframe
pd.options.display.float_format = '{:.2f}'.format   # pandas: para todos os números aparecerem com duas casas decimais

# Cria uma instância da API
api = cartolafc.Api(attempts=5)

In [52]:
# Carregar o arquivo CSV
df_times = pd.read_excel("times.xlsx")

# Ver os dados carregados
display(df_times.head())

,Nome
0,A Lenda Super Vascão f.c
1,A Lenda Super Vasco F.c
2,Analove10 ITAQUI GRANDE!!
3,BordonFC
4,BORGES ITAQUI F.C.


### Buscar IDs dos times no Cartola

In [53]:
nomes_times = ['FBC Colorado', 'Texas Club 2025', 'Real SCI', 'Gig@ntte', 'PraiaBravaFC', 'OlhaEleAiF.C!', 'Gremiomaniasm', 'Sport Clube PAIM', 'PUXE FC', 'RS Expressões da Arte',
               'ZIVI FC', 'O clube do povo Itaqui/Rss', 'F.C. Rei Das Copas', 'Rolo Compressor ZN', 'Taura da Fronteira FCIII', 'Eleis-Itaqui', 'KING LEONN', 'Laranjja Mecannica', 
               'Fedato Futebol Clube', 'Perronee F.C', 'Pity10', 'pra sempre imortal fc', 'RHANKA DENTY FC25', 'TEAM LOPES 99', 'pura bucha /botafogo', 'cartola scheuer', 
               'Analove10 ITAQUI GRANDE!!', 'DM Studio', 'lsauer fc', 'VASCO MARTINS FC', 'KP JUV.', 'BORGES ITAQUI F.C.', 'Profit Soccer', 'Tabajara de Inhaua FC2', 'TIGRE LEON', 
               'S.E.R. GRILLO', 'seralex', 'E.C. Bororé', 'MAFRA MARTINS FC', 'BordonFC', 'Tatols Beants F.C', 'FIGUEIRA DA ILHA', 'MauHumor F.C.', 'A Lenda Super Vascão f.c', 
               'TATITTA FC', 'HS SPORTS F.C', 'Dom Camillo68', 'mercearia Estrela', 'CosmoCity ZO', 'clarinvalau fc', 'Grêmio imortal 37', 'SERGRILLO', 'Super Vasco f.c', 
               'A Lenda Super Vasco F.c', 'Paulo Virgili FC', 'CALOMBO ITAQUI RS', 'Luis lemes inter', 'emer jr fc', 'Cril Futebol Club', 'ITAQUI F.C.', 'TORRESMO COM PINGA', 
               'Lá do Itaqui', 'FC Los Castilho', 'KillerColorado', "Atlético Colorado 2021", "Time do S.A.P.O", "SISO FC25", "Grêmio imortal 37", "BORGES ITAQUI F.C."] #, 'Remo Santo Ângelo'

# Dicionário para armazenar os IDs dos times
ids_times = {}

# Função robusta para buscar ID do time por nome
from difflib import get_close_matches

def buscar_id_time(nome_time):
    try:
        times = api.times(query=nome_time)

        # 🛡️ Proteção caso a resposta seja uma string (ex: erro HTML ou mensagem)
        if not isinstance(times, list):
            print(f"⚠️ Resposta inesperada para '{nome_time}':", times)
            return None

        nomes_api = [time.nome for time in times]

        # Comparação flexível (removendo acentos, pontos, etc)
        nome_base = nome_time.lower().replace(".", "").replace("fc", "").replace("f.c", "").strip()
        nomes_api_base = [n.lower().replace(".", "").replace("fc", "").replace("f.c", "").strip() for n in nomes_api]

        nome_proximo = get_close_matches(nome_base, nomes_api_base, n=1, cutoff=0.6)

        if nome_proximo:
            idx = nomes_api_base.index(nome_proximo[0])
            return times[idx].id

    except Exception as e:
        print(f"❌ Erro ao buscar ID para o time '{nome_time}': {e}")
    return None


# Buscar IDs automaticamente, sem sobrescrever os que já estiverem no dicionário
nao_encontrados = []

for nome in nomes_times:
    if nome not in ids_times:  # Protege os manuais
        time_id = buscar_id_time(nome)
        if time_id:
            ids_times[nome] = time_id
        else:
            nao_encontrados.append(nome)

# ➕ Correções manuais
# ids_times["BORGES ITAQUI F.C."] = 3914981
# ids_times["Time do S.A.P.O"] = 12345678  # Substitua pelo ID real

# Mostrar resultado
print("IDs encontrados:", len(ids_times))
print("Times não encontrados automaticamente:", nao_encontrados)

IDs encontrados: 67
Times não encontrados automaticamente: []


### Gerar DataFrame com nome do time, ID e URL

In [54]:
# Gerar DataFrame com nome do time, ID e URL
df_urls = pd.DataFrame([
    {"Nome do Time": nome, "ID do Time": time_id, "Link do Time": f"https://cartola.globo.com/#!/time/{time_id}"}
    for nome, time_id in ids_times.items()
])

# Caminho e nome do arquivo Excel
caminho_excel = "links_times_cartola_liga_serie_A.xlsx"

# Salvar o DataFrame em Excel
df_urls.to_excel(caminho_excel, index=False)

print(f"✅ Arquivo salvo com sucesso: {caminho_excel}")


# Exibir como tabela
display(df_urls.head(30))

✅ Arquivo salvo com sucesso: links_times_cartola_liga_serie_A.xlsx


,Nome do Time,ID do Time,Link do Time
0,FBC Colorado,186283,https://cartola.globo.com/#!/time/186283
1,Texas Club 2025,1273719,https://cartola.globo.com/#!/time/1273719
2,Real SCI,3246608,https://cartola.globo.com/#!/time/3246608
3,Gig@ntte,18421230,https://cartola.globo.com/#!/time/18421230
4,PraiaBravaFC,47546604,https://cartola.globo.com/#!/time/47546604
5,OlhaEleAiF.C!,3708025,https://cartola.globo.com/#!/time/3708025
6,Gremiomaniasm,528730,https://cartola.globo.com/#!/time/528730
7,Sport Clube PAIM,1148959,https://cartola.globo.com/#!/time/1148959
8,PUXE FC,3447341,https://cartola.globo.com/#!/time/3447341
9,RS Expressões da Arte,896842,https://cartola.globo.com/#!/time/896842


### Gerar o dicionário ID -> Nome do Time

In [55]:
# Gerar o dicionário ID -> Nome do Time
nomes_por_id = dict(zip(df_urls["ID do Time"], df_urls["Nome do Time"]))

# Mostrar parte do dicionário
dict(list(nomes_por_id.items())[:5])

{186283: 'FBC Colorado',
 1273719: 'Texas Club 2025',
 3246608: 'Real SCI',
 18421230: 'Gig@ntte',
 47546604: 'PraiaBravaFC'}

In [56]:
def campeonato_comecou(api, ids_times):
    """Verifica se o campeonato já começou observando a pontuação na 1ª rodada."""
    for time_id in ids_times.values():
        try:
            pontuacao = api.time(time_id=time_id, rodada=1).ultima_pontuacao
            if pontuacao is not None:
                return True
        except cartolafc.errors.CartolaFCError:
            continue
    return False

def obter_pontuacao_por_rodada(api, time_id, rodada_atual):
    """Obtém a pontuação do time em cada rodada até a rodada atual."""
    pontuacoes = {}
    for rodada in range(1, rodada_atual):
        try:
            time_rodada = api.time(time_id=time_id, rodada=rodada)
            pontuacoes[rodada] = time_rodada.ultima_pontuacao
        except cartolafc.errors.CartolaFCError as e:
            print(f"Erro ao acessar pontuação da rodada {rodada} para o time {time_id}: {e}")
            pontuacoes[rodada] = None
    return pontuacoes

def gerar_df_pontuacoes(api, ids_times):
    rodada_atual = api.mercado().rodada_atual
    total_rodadas = 38

    if not campeonato_comecou(api, ids_times):
        print("📌 O campeonato ainda não começou. Criando estrutura com placeholders.")
        df = pd.DataFrame(index=ids_times.keys(), columns=[f'Rodada {i}' for i in range(1, total_rodadas + 1)])
        df[:] = 0
    else:
        df = pd.DataFrame()
        for nome, time_id in ids_times.items():
            pontuacoes = obter_pontuacao_por_rodada(api, time_id, rodada_atual)
            df[nome] = pd.Series(pontuacoes)
        df = df.transpose()
        df.columns = [f'Rodada {i}' for i in range(1, rodada_atual)]
        df.loc['Lider_Rodada'] = df.idxmax()
    
    return df

In [57]:
ids_times = {v: k for k, v in nomes_por_id.items()}

df_pontuacoes = gerar_df_pontuacoes(api, ids_times)

display(df_pontuacoes.head(20))

,Rodada 1,Rodada 2,Rodada 3,Rodada 4,Rodada 5
FBC Colorado,64.00,72.69,81.52,78.62,69.38
Texas Club 2025,67.69,85.99,76.92,79.86,71.32
Real SCI,104.09,75.69,87.02,99.66,82.42
Gig@ntte,100.34,86.39,86.80,79.58,73.01
PraiaBravaFC,96.39,96.19,87.05,81.06,94.26
OlhaEleAiF.C!,96.19,102.99,54.12,98.27,92.73
Gremiomaniasm,95.23,59.22,95.55,85.67,86.03
Sport Clube PAIM,93.85,67.39,85.52,91.70,86.48
PUXE FC,84.69,95.29,86.95,85.01,73.92
RS Expressões da Arte,83.99,92.82,68.52,77.28,57.92


### Função para definir a classificação dos times

In [58]:
def classificacao_por_grupo(df_rodadas, df_pontuacoes):
    """
    Classificação dos grupos com base nos confrontos e nas pontuações do Cartola.

    Retorna:
    - df_resultado: classificação geral
    - df_resultado_por_grupo: dicionário com classificação separada por grupo
    """
    df_pontuacoes_times = df_pontuacoes.drop(index='Lider_Rodada', errors='ignore')
    estatisticas = {}

    for _, confronto in df_rodadas.iterrows():
        rodada = confronto["Rodada"]
        mandante = confronto["Mandante_Nome"]
        visitante = confronto["Visitante_Nome"]
        grupo = confronto["Grupo"]
        coluna_rodada = f"Rodada {rodada}"

        if mandante not in df_pontuacoes_times.index or visitante not in df_pontuacoes_times.index:
            continue
        if coluna_rodada not in df_pontuacoes_times.columns:
            continue

        pontos_mandante = df_pontuacoes_times.at[mandante, coluna_rodada]
        pontos_visitante = df_pontuacoes_times.at[visitante, coluna_rodada]

        if pd.isnull(pontos_mandante) or pd.isnull(pontos_visitante):
            continue

        for time in [mandante, visitante]:
            if grupo not in estatisticas:
                estatisticas[grupo] = {}
            if time not in estatisticas[grupo]:
                estatisticas[grupo][time] = {
                    "Pontos": 0, "Vitórias": 0, "Empates": 0, "Derrotas": 0,
                    "Total_Cartola": 0, "Cartola_Sofrido": 0
                }

        # Atualizar estatísticas do jogo
        estatisticas[grupo][mandante]["Total_Cartola"] += pontos_mandante
        estatisticas[grupo][mandante]["Cartola_Sofrido"] += pontos_visitante

        estatisticas[grupo][visitante]["Total_Cartola"] += pontos_visitante
        estatisticas[grupo][visitante]["Cartola_Sofrido"] += pontos_mandante

        if pontos_mandante > pontos_visitante:
            estatisticas[grupo][mandante]["Pontos"] += 3
            estatisticas[grupo][mandante]["Vitórias"] += 1
            estatisticas[grupo][visitante]["Derrotas"] += 1
        elif pontos_mandante < pontos_visitante:
            estatisticas[grupo][visitante]["Pontos"] += 3
            estatisticas[grupo][visitante]["Vitórias"] += 1
            estatisticas[grupo][mandante]["Derrotas"] += 1
        else:
            estatisticas[grupo][mandante]["Pontos"] += 1
            estatisticas[grupo][visitante]["Pontos"] += 1
            estatisticas[grupo][mandante]["Empates"] += 1
            estatisticas[grupo][visitante]["Empates"] += 1

    # Montar DataFrame final
    df_resultado = pd.concat([
        pd.DataFrame({
            "Grupo": grupo,
            "Nome do Time": list(times.keys()),
            "Pontos": [stats["Pontos"] for stats in times.values()],
            "Vitórias": [stats["Vitórias"] for stats in times.values()],
            "Empates": [stats["Empates"] for stats in times.values()],
            "Derrotas": [stats["Derrotas"] for stats in times.values()],
            "Total Cartola": [stats["Total_Cartola"] for stats in times.values()],
            "Cartola Sofrido": [stats["Cartola_Sofrido"] for stats in times.values()],
            "Saldo Cartola": [stats["Total_Cartola"] - stats["Cartola_Sofrido"] for stats in times.values()
            ]
        })
        for grupo, times in estatisticas.items()
    ], ignore_index=True)


    df_resultado = df_resultado.sort_values(
        by=["Grupo", "Pontos", "Total Cartola"],
        ascending=[True, False, False]
    )

    df_resultado["Posição"] = df_resultado.groupby("Grupo")\
    .cumcount() + 1

    df_resultado_por_grupo = {
        grupo: df_resultado[df_resultado["Grupo"] == grupo] for grupo in df_resultado["Grupo"].unique()
    }

    return df_resultado, df_resultado_por_grupo

In [59]:
# 1. Carregar confrontos
df_confrontos = pd.read_csv("confrontos_serie_A.csv")

# 2. Renomear colunas se necessário
df_confrontos.rename(columns={
    "Time A": "Mandante_Nome",
    "Time B": "Visitante_Nome"
}, inplace=True)

# 3. Adicionar coluna de grupo fixo
df_confrontos["Grupo"] = "Série A"

display(df_confrontos.head())

# 4. Calcular classificação
df_classificacao, _ = classificacao_por_grupo(df_confrontos, df_pontuacoes)

# 5. Ver resultado
display(df_classificacao.head())

,Rodada,Confronto,Mandante_Nome,Visitante_Nome,ID A,ID B,Grupo
0,1,1,FBC Colorado,Tatols Beants F.C,6,15,Série A
1,1,2,BORGES ITAQUI F.C.,Time do S.A.P.O,3,17,Série A
2,1,3,Fedato Futebol Clube,OlhaEleAiF.C!,7,10,Série A
3,1,4,Atlético Colorado 2021,Texas Club 2025,2,16,Série A
4,1,5,Tabajara de Inhaua FC2,Analove10 ITAQUI GRANDE!!,14,1,Série A


,Grupo,Nome do Time,Pontos,Vitórias,Empates,Derrotas,Total Cartola,Cartola Sofrido,Saldo Cartola,Posição
5,Série A,OlhaEleAiF.C!,12,4,0,1,444.30,390.73,53.57,1
18,Série A,Grêmio imortal 37,12,4,0,1,429.02,388.02,41.00,2
9,Série A,Analove10 ITAQUI GRANDE!!,12,4,0,1,422.11,356.43,65.68,3
11,Série A,Gremiomaniasm,12,4,0,1,421.70,378.53,43.17,4
19,Série A,SISO FC25,12,4,0,1,415.98,391.23,24.75,5


In [60]:
# 1. Times únicos nos confrontos
times_confrontos = pd.unique(df_confrontos[["Mandante_Nome", "Visitante_Nome"]].values.ravel())

# 2. Times únicos na classificação final
times_classificados = df_classificacao["Nome do Time"].unique()

# 3. Ver quem está nos confrontos mas não foi classificado
faltando = set(times_confrontos) - set(times_classificados)

print("❌ Times que estão nos confrontos, mas faltam na classificação:")
print(faltando)

# df_pontuacoes.loc[["Time do S.A.P.O", "BORGES ITAQUI F.C."]]


❌ Times que estão nos confrontos, mas faltam na classificação:
set()


In [61]:
# Exportar resultado em CSV
df_classificacao.to_csv("classificacao_serie_a.csv", index=False)

# Carregar o arquivo CSV enviado
df_classificacao = pd.read_csv("classificacao_serie_a.csv")

# Renomear colunas para os nomes usados no JavaScript
df_classificacao.rename(columns={
    "Grupo": "grupo",
    "Nome do Time": "nome",
    "Pontos": "pontos",
    "Vitórias": "vitorias",
    "Empates": "empates",
    "Derrotas": "derrotas",
    "Total Cartola": "totalCartola",
    "Cartola Sofrido": "cartolaSofrido",
    "Saldo Cartola": "saldoCartola",
    "Posição": "posicao"
}, inplace=True)

# Agrupar por grupo (apenas Série A nesse caso)
classificacao_por_grupo = {}
for grupo, dados in df_classificacao.groupby("grupo"):
    classificacao_por_grupo[grupo] = dados.drop(columns="grupo").to_dict(orient="records")

# Salvar como arquivo JavaScript
caminho_saida = "classificacao_serie_A.js"
with open(caminho_saida, "w", encoding="utf-8") as f:
    f.write(f"const classificacaoSerieA = {json.dumps(classificacao_por_grupo, indent=2, ensure_ascii=False)};")

caminho_saida

'classificacao_serie_A.js'

In [62]:
import pandas as pd
import json

# Carregar CSV
df_confrontos = pd.read_csv("confrontos_serie_A.csv")

# Limpar espaços nas colunas
df_confrontos.columns = df_confrontos.columns.str.strip()

# Renomear colunas
df_confrontos.rename(columns={
    "Rodada": "rodada",
    "Confronto": "confronto",
    "Time A": "mandante_nome",
    "Time B": "visitante_nome",
    "ID A": "mandante_id",
    "ID B": "visitante_id"
}, inplace=True)

# Adicionar grupo fixo
df_confrontos["grupo"] = "Série A"

# Gerar estrutura para JS
confrontos_formatado = []
for _, row in df_confrontos.iterrows():
    confronto = {
        "rodada": int(row["rodada"]),
        "confronto": int(row["confronto"]),
        "grupo": row["grupo"],
        "mandante": {"id": row["mandante_id"], "nome": row["mandante_nome"]},
        "visitante": {"id": row["visitante_id"], "nome": row["visitante_nome"]}
    }
    confrontos_formatado.append(confronto)

# Salvar como arquivo JS
js_output = f"const confrontosFase1 = {json.dumps(confrontos_formatado, indent=2, ensure_ascii=False)};"
with open("confrontos_serie_A.js", "w", encoding="utf-8") as f:
    f.write(js_output)

print("✅ Arquivo confrontos_serie_A.js gerado com sucesso!")


✅ Arquivo confrontos_serie_A.js gerado com sucesso!


In [63]:
# Carregar o arquivo CSV enviado
df_confrontos = pd.read_csv("confrontos_serie_A.csv")

display(df_confrontos.head())

,Rodada,Confronto,Time A,Time B,ID A,ID B
0,1,1,FBC Colorado,Tatols Beants F.C,6,15
1,1,2,BORGES ITAQUI F.C.,Time do S.A.P.O,3,17
2,1,3,Fedato Futebol Clube,OlhaEleAiF.C!,7,10
3,1,4,Atlético Colorado 2021,Texas Club 2025,2,16
4,1,5,Tabajara de Inhaua FC2,Analove10 ITAQUI GRANDE!!,14,1


In [64]:
def gerar_resultados_serie_a(df_confrontos, df_pontuacoes, grupo="Série A"):
    """
    Gera um DataFrame com os resultados dos confrontos da Série A por rodada.
    """
    resultados = []

    for _, row in df_confrontos.iterrows():
        rodada = row["Rodada"]
        mandante = row["Time A"]
        visitante = row["Time B"]

        pontos_mandante = df_pontuacoes.get(f"Rodada {rodada}", {}).get(mandante, None)
        pontos_visitante = df_pontuacoes.get(f"Rodada {rodada}", {}).get(visitante, None)

        resultados.append({
            "grupo": grupo,
            "rodada": rodada,
            "mandante": {
                "nome": mandante,
                "pontos": pontos_mandante
            },
            "visitante": {
                "nome": visitante,
                "pontos": pontos_visitante
            }
        })

    return pd.DataFrame(resultados)


In [65]:
df_resultados = gerar_resultados_serie_a(df_confrontos, df_pontuacoes)

# Exportar para .js
import json

with open("resultados_serie_A.js", "w", encoding="utf-8") as f:
    f.write("const resultadosFase1 = ")
    f.write(json.dumps(df_resultados.to_dict(orient="records"), indent=2, ensure_ascii=False))
    f.write(";")

print("✅ Arquivo resultados_serie_A.js gerado com sucesso.")


✅ Arquivo resultados_serie_A.js gerado com sucesso.


# Análises de Pontuações de Jogadores

## Obter Pontuações por Rodada

In [66]:
# Obter a rodada atual do mercado
rodada_atual = api.mercado().rodada_atual

# Função para verificar se o campeonato já começou (há pontuação na rodada 1)
def campeonato_comecou(ids_times):
    for time_id in ids_times.values():
        try:
            pontuacao = api.time(time_id=time_id, rodada=1).ultima_pontuacao
            if pontuacao is not None:
                return True  # Campeonato já começou
        except cartolafc.errors.CartolaFCError:
            continue
    return False  # Nenhuma pontuação encontrada

# Verifica se o campeonato começou
if not campeonato_comecou(ids_times):
    print("📌 O campeonato ainda não começou. Criando estrutura com placeholders.")

    total_rodadas = 38  # Número total de rodadas do campeonato
    df_pontuacoes = pd.DataFrame(index=ids_times.keys(), columns=[f'Rodada {i}' for i in range(1, total_rodadas + 1)])
    df_pontuacoes[:] = 0  # ou None se preferir

else:
    # Função para obter a pontuação de um time em todas as rodadas
    def obter_pontuacao_por_rodada(time_id, rodada_atual):
        pontuacoes = {}
        for rodada in range(1, rodada_atual):
            try:
                time_rodada = api.time(time_id=time_id, rodada=rodada)
                pontuacoes[rodada] = time_rodada.ultima_pontuacao
            except cartolafc.errors.CartolaFCError as e:
                print(f"Erro ao acessar a pontuação da rodada {rodada} para o time {time_id}: {e}")
                pontuacoes[rodada] = None
        return pontuacoes

    # DataFrame para armazenar as pontuações por rodada
    df_pontuacoes = pd.DataFrame()

    # Obter as pontuações por rodada para cada time
    for nome, time_id in ids_times.items():
        pontuacoes = obter_pontuacao_por_rodada(time_id, rodada_atual)
        df_pontuacoes[nome] = pd.Series(pontuacoes)

    # Transpor o DataFrame para ter rodadas como colunas
    df_pontuacoes = df_pontuacoes.transpose()

    # Renomear as colunas para representar as rodadas
    df_pontuacoes.columns = [f'Rodada {i}' for i in range(1, rodada_atual)]

    # Adicionar a linha "Lider_Rodada"
    lider_rodada = df_pontuacoes.idxmax()
    # df_pontuacoes.loc['Lider_Rodada'] = lider_rodada

df_pontuacoes = df_pontuacoes.sort_values(by="Rodada 1", ascending=False)

# Exibir o dataframe
display(df_pontuacoes)

# Salvar o dataframe em um arquivo Excel
df_pontuacoes.to_excel('pontuacoes_por_rodada_serie_A.xlsx')

# _______________________________________________________________________________________________________________________________________________

# Transformar o DataFrame em dicionário: { "Rodada 1": { "Time A": 50.12, ... }, ... }
pontuacoes_dict = df_pontuacoes.transpose().to_dict()

# Converter para formato JS
conteudo_js = f"const pontuacoesPorRodada = {json.dumps(pontuacoes_dict, indent=2, ensure_ascii=False)};"

# Salvar no arquivo .js
with open("pontuacoes_por_rodada_serie_A.js", "w", encoding="utf-8") as f:
    f.write(conteudo_js)

print("✅ Arquivo pontuacoes_por_rodada_serie_A.js salvo com sucesso!")



,Rodada 1,Rodada 2,Rodada 3,Rodada 4,Rodada 5
Real SCI,104.09,75.69,87.02,99.66,82.42
Gig@ntte,100.34,86.39,86.80,79.58,73.01
PraiaBravaFC,96.39,96.19,87.05,81.06,94.26
OlhaEleAiF.C!,96.19,102.99,54.12,98.27,92.73
Gremiomaniasm,95.23,59.22,95.55,85.67,86.03
...,...,...,...,...,...
CALOMBO ITAQUI RS,56.38,80.62,47.35,102.56,56.88
KillerColorado,56.14,103.22,63.90,99.16,69.15
Luis lemes inter,53.64,80.97,68.55,67.58,74.91
emer jr fc,50.03,60.88,56.00,72.37,54.51


✅ Arquivo pontuacoes_por_rodada_serie_A.js salvo com sucesso!


## Obter Escalação de um  Time em uma determinada Rodada

In [67]:
# Função para obter a escalação de um time em uma rodada específica, incluindo o clube do jogador
def obter_escalacao_time(time_id, rodada):
    try:
        time_rodada = api.time(time_id=time_id, rodada=rodada)
        if not time_rodada.atletas:
            print(f"⚠️ Nenhum jogador encontrado para o time {time_id} na rodada {rodada}. A rodada pode ainda não ter ocorrido ou os dados ainda não foram liberados.")
            return []
        jogadores = [
            {
                'Nome': atleta.apelido,
                'Posição': atleta.posicao.nome,
                'Pontos': atleta.pontos,
                'Clube': atleta.clube.nome
            }
            for atleta in time_rodada.atletas
        ]
        return jogadores
    except cartolafc.errors.CartolaFCError as e:
        print(f"Erro ao acessar a escalação da rodada {rodada} para o time {time_id}: {e}")
        return []

# Parâmetros de entrada
rodada_especifica = 4  # Substitua pela rodada desejada
nome_time = 'PraiaBravaFC'  # Substitua pelo nome do time desejado

# Verificação se o time está no dicionário
if nome_time not in ids_times:
    print(f"❌ O time '{nome_time}' não foi encontrado no dicionário de IDs.")
else:
    time_cartola = ids_times[nome_time]
    time_escalacao = obter_escalacao_time(time_cartola, rodada_especifica)

    if time_escalacao:
        df_escalacao = pd.DataFrame(time_escalacao)
        # df_escalacao.sort_values(by='Posição', inplace=True)  # Ou por 'Pontos' se preferir

        df_escalacao_pontuacao = df_escalacao['Pontos'].sum()

        print(f'✅ Time do Cartola: {nome_time}')
        print(f'📊 Pontuação na Rodada {rodada_especifica}: {df_escalacao_pontuacao:.2f}')
        display(df_escalacao)
    else:
        print("⚠️ Escalação vazia ou não disponível.")



✅ Time do Cartola: PraiaBravaFC
📊 Pontuação na Rodada 4: 79.31


,Nome,Posição,Pontos,Clube
0,Gabriel Brazão,Goleiro,10.20,SAN
1,Kaiki Bruno,Lateral,7.90,CRU
2,Escobar,Lateral,10.70,SAN
3,Jonathan Jesus,Zagueiro,0.00,CRU
4,Léo Ortiz,Zagueiro,0.00,FLA
5,Arrascaeta,Meia,18.80,FLA
6,Alan Patrick,Meia,5.50,INT
7,Neymar,Meia,0.50,SAN
8,Enner Valencia,Atacante,3.90,INT
9,Yuri Alberto,Atacante,3.50,COR


## Obter Escalações Gerais

In [68]:
import time  # opcional, para colocar pausa entre requisições

# Função para obter escalações de todos os times em todas as rodadas
def obter_escalacoes_gerais(ids_times, rodada_atual):
    escalacoes_gerais = []

    for nome_time, time_id in ids_times.items():
        print(f"🔄 Coletando escalações para o time: {nome_time}")
        for rodada in range(1, rodada_atual + 1):
            escalação = obter_escalacao_time(time_id, rodada)

            if not escalação:
                print(f"⚠️ Rodada {rodada}: sem escalação disponível para {nome_time}.")
                continue

            for jogador in escalação:
                jogador['Time Cartola'] = nome_time
                jogador['Rodada'] = rodada
                escalacoes_gerais.append(jogador)

            # Opcional: pausa para evitar sobrecarga de requisições (API friendly)
            time.sleep(0.2)

    return escalacoes_gerais

# Obter escalações gerais
escalacoes_gerais = obter_escalacoes_gerais(ids_times, rodada_atual)

# Armazenar escalações em um DataFrame
df_escalacoes = pd.DataFrame(escalacoes_gerais)

if df_escalacoes.empty:
    print("⚠️ Nenhuma escalação disponível até o momento. O DataFrame está vazio.")
else:
    # Organizar colunas (opcional)
    colunas_ordenadas = ['Rodada', 'Time Cartola', 'Nome', 'Posição', 'Clube', 'Pontos']
    df_escalacoes = df_escalacoes[colunas_ordenadas]

    # Salvar o DataFrame em um arquivo Excel
    df_escalacoes.to_excel('escalacoes_gerais_serie_A.xlsx', index=False)

    # Visualizar primeiras linhas
    display(df_escalacoes.head())

# _________________________________________________________________________________________________________________________________

# Estrutura: { "Rodada 1": [ { jogador1 }, { jogador2 }, ... ], "Rodada 2": [...] }
escalacoes_dict = {}

for _, row in df_escalacoes.iterrows():
    rodada = f"Rodada {int(row['Rodada'])}"
    jogador_info = {
        "time": row["Time Cartola"],
        "nome": row["Nome"],
        "posicao": row["Posição"],
        "clube": row["Clube"],
        "pontos": row["Pontos"]
    }
    escalacoes_dict.setdefault(rodada, []).append(jogador_info)

# Converter para JavaScript
conteudo_js = f"const escalacoesGerais = {json.dumps(escalacoes_dict, indent=2, ensure_ascii=False)};"

# Salvar o arquivo
with open("escalacoes_gerais_serie_A.js", "w", encoding="utf-8") as f:
    f.write(conteudo_js)

print("✅ Arquivo escalacoes_gerais_serie_A.js salvo com sucesso!")


🔄 Coletando escalações para o time: FBC Colorado
🔄 Coletando escalações para o time: Texas Club 2025
🔄 Coletando escalações para o time: Real SCI
🔄 Coletando escalações para o time: Gig@ntte
🔄 Coletando escalações para o time: PraiaBravaFC
🔄 Coletando escalações para o time: OlhaEleAiF.C!
🔄 Coletando escalações para o time: Gremiomaniasm
🔄 Coletando escalações para o time: Sport Clube PAIM
🔄 Coletando escalações para o time: PUXE FC
🔄 Coletando escalações para o time: RS Expressões da Arte
🔄 Coletando escalações para o time: ZIVI FC
🔄 Coletando escalações para o time: O clube do povo Itaqui/Rss
🔄 Coletando escalações para o time: F.C. Rei Das Copas
🔄 Coletando escalações para o time: Rolo Compressor ZN
🔄 Coletando escalações para o time: Taura da Fronteira FCIII
🔄 Coletando escalações para o time: Eleis-Itaqui
🔄 Coletando escalações para o time: KING LEONN
🔄 Coletando escalações para o time: Laranjja Mecannica
🔄 Coletando escalações para o time: Fedato Futebol Clube
🔄 Coletando escalaç

,Rodada,Time Cartola,Nome,Posição,Clube,Pontos
0,1,FBC Colorado,Weverton,Goleiro,PAL,11.50
1,1,FBC Colorado,Renê,Lateral,FLU,5.00
2,1,FBC Colorado,Juninho Capixaba,Lateral,RBB,7.40
3,1,FBC Colorado,Sabino,Zagueiro,SAO,7.30
4,1,FBC Colorado,Léo Ortiz,Zagueiro,FLA,0.60


✅ Arquivo escalacoes_gerais_serie_A.js salvo com sucesso!


## Função para listar os jogadores mais escalados por posição

In [69]:
# Função para listar os jogadores mais escalados por posição
def listar_jogadores_mais_escalados(df, n_goleiros=3, n_zagueiros=4, n_laterais=4, n_meias=5, n_atacantes=5):
    if df.empty:
        print("⚠️ O DataFrame está vazio. Nenhuma escalação disponível ainda.")
        return pd.DataFrame()

    # Agrupar por nome, posição e clube, somando os pontos e contando a quantidade de vezes escalados
    df_agrupado = df.groupby(['Nome', 'Posição', 'Clube'], as_index=False).agg(
        Pontos=('Pontos', 'sum'),
        Quantidade=('Nome', 'size')
    )

    # Filtrar e ordenar para cada posição
    goleiros = df_agrupado[df_agrupado['Posição'] == 'Goleiro'].sort_values(by='Quantidade', ascending=False).head(n_goleiros)
    zagueiros = df_agrupado[df_agrupado['Posição'] == 'Zagueiro'].sort_values(by='Quantidade', ascending=False).head(n_zagueiros)
    laterais = df_agrupado[df_agrupado['Posição'] == 'Lateral'].sort_values(by='Quantidade', ascending=False).head(n_laterais)
    meias = df_agrupado[df_agrupado['Posição'] == 'Meia'].sort_values(by='Quantidade', ascending=False).head(n_meias)
    atacantes = df_agrupado[df_agrupado['Posição'] == 'Atacante'].sort_values(by='Quantidade', ascending=False).head(n_atacantes)

    # Concatenar os resultados em um único dataframe
    jogadores_mais_escalados = pd.concat([goleiros, zagueiros, laterais, meias, atacantes])

    # colunas = ['Nome', 'Posição', 'Clube', 'Quantidade', 'Pontos']
    colunas = ['Nome', 'Posição', 'Clube', 'Quantidade']
    jogadores_mais_escalados = jogadores_mais_escalados[colunas]

    return jogadores_mais_escalados

# Execução protegida com verificação
jogadores_mais_escalados = listar_jogadores_mais_escalados(df_escalacoes)

if not jogadores_mais_escalados.empty:
    print("📋 Jogadores mais escalados por posição:")
    display(jogadores_mais_escalados)


📋 Jogadores mais escalados por posição:


,Nome,Posição,Clube,Quantidade
191,Matheus Donelli,Goleiro,COR,75
17,Anthoni,Goleiro,INT,67
271,Weverton,Goleiro,PAL,54
177,Léo Ortiz,Zagueiro,FLA,179
95,Félix Torres,Zagueiro,COR,56
145,Junior Alonso,Zagueiro,CAM,52
131,Jair,Zagueiro,BOT,41
144,Juninho Capixaba,Lateral,RBB,144
16,Angileri,Lateral,COR,69
270,Wesley,Lateral,FLA,68


## Função para listar os times que mais escalaram um determinado jogador

In [70]:
# Função para listar os times que mais escalaram um determinado jogador
def listar_times_que_mais_escalaram(df, nome_jogador, clube_jogador):
    if df.empty:
        print("⚠️ O DataFrame está vazio. Nenhum dado disponível para análise.")
        return pd.DataFrame()
    
    df_jogador = df[(df['Nome'] == nome_jogador) & (df['Clube'] == clube_jogador)]

    if df_jogador.empty:
        print(f"⚠️ Nenhuma escalação encontrada para o jogador '{nome_jogador}' do clube '{clube_jogador}'.")
        return pd.DataFrame()

    df_times_que_mais_escalaram = df_jogador.groupby('Time Cartola', as_index=False).agg(
        Quantidade=('Nome', 'size')
    ).sort_values(by='Quantidade', ascending=False)
    
    return df_times_que_mais_escalaram

# Exemplo de uso
nome_jogador = "Raphael Veiga"
clube_jogador = "Palmeiras"

times_que_mais_escalaram = listar_times_que_mais_escalaram(df_escalacoes, nome_jogador, clube_jogador)

if not times_que_mais_escalaram.empty:
    print(f"📊 Times que mais escalaram o jogador {nome_jogador} ({clube_jogador}):")
    display(times_que_mais_escalaram)


⚠️ Nenhuma escalação encontrada para o jogador 'Raphael Veiga' do clube 'Palmeiras'.


## Função para Listar Jogadores Agrupados por 'Time Cartola'  e Posição

In [71]:
# Função para listar todos os jogadores escalados por um determinado time, agrupados por posição, nome e clube
def listar_jogadores_escalados_por_time_e_agrupados(df, nome_time, posicao=None):
    if df.empty:
        print("⚠️ O DataFrame está vazio. Nenhum dado disponível para análise.")
        return pd.DataFrame()
    
    if nome_time not in df['Time Cartola'].unique():
        print(f"⚠️ O time '{nome_time}' não possui escalações registradas.")
        return pd.DataFrame()

    df_time = df[df['Time Cartola'] == nome_time]
    
    if posicao:
        df_time = df_time[df_time['Posição'] == posicao]
        if df_time.empty:
            print(f"⚠️ Nenhum jogador na posição '{posicao}' encontrado para o time '{nome_time}'.")
            return pd.DataFrame()

    df_agrupado = df_time.groupby(['Nome', 'Posição', 'Clube'], as_index=False).agg(
        Pontos=('Pontos', 'sum'),
        Quantidade=('Nome', 'size')
    )

    df_agrupado = df_agrupado.sort_values(by='Quantidade', ascending=False)

    return df_agrupado

# Exemplo de uso
nome_time = "PraiaBravaFC"
posicao = 'Zagueiro'  # Substitua por outra posição ou deixe como None

jogadores_escalados_agrupados = listar_jogadores_escalados_por_time_e_agrupados(df_escalacoes, nome_time, posicao)

if not jogadores_escalados_agrupados.empty:
    display(jogadores_escalados_agrupados.head(20))


,Nome,Posição,Clube,Pontos,Quantidade
5,Léo Ortiz,Zagueiro,FLA,10.40,4
1,Ferraresi,Zagueiro,SAO,-0.60,2
0,Alan Franco,Zagueiro,SAO,9.20,1
2,Jonathan Jesus,Zagueiro,CRU,0.00,1
3,Junior Alonso,Zagueiro,CAM,0.80,1
4,Kanu,Zagueiro,BAH,1.50,1
6,Micael,Zagueiro,PAL,4.70,1
7,Rogel,Zagueiro,INT,6.20,1


### Função para listar os jogadores mais escalados por posição para um determinado time do Cartola

In [72]:
# Função para listar os jogadores mais escalados por posição para um determinado time do Cartola
def listar_jogadores_mais_escalados_por_posicao(df, nome_time, top_n=3):
    if df.empty:
        print("⚠️ O DataFrame está vazio. Nenhum dado disponível.")
        return pd.DataFrame()

    if nome_time not in df['Time Cartola'].unique():
        print(f"⚠️ O time '{nome_time}' não possui escalações registradas.")
        return pd.DataFrame()

    df_time = df[df['Time Cartola'] == nome_time].copy()

    df_agrupado = df_time.groupby(['Posição', 'Nome', 'Clube'], as_index=False).agg(
        Pontos=('Pontos', 'sum'),
        Quantidade=('Nome', 'size')
    )

    # Ordenar por posição e quantidade
    df_agrupado = df_agrupado.sort_values(by=['Posição', 'Quantidade'], ascending=[True, False])

    # Selecionar os top N jogadores para cada posição
    df_top_n_por_posicao = df_agrupado.groupby('Posição').head(top_n).copy()

    # Definir a ordem específica das posições
    ordem_posicoes = ['Goleiro', 'Lateral', 'Zagueiro', 'Meia', 'Atacante', 'Técnico']
    df_top_n_por_posicao['Posição'] = pd.Categorical(df_top_n_por_posicao['Posição'], categories=ordem_posicoes, ordered=True)

    # Ordenar conforme a ordem definida
    df_top_n_por_posicao = df_top_n_por_posicao.sort_values(by=['Posição', 'Quantidade'], ascending=[True, False])

    return df_top_n_por_posicao

# Função para calcular a pontuação total e média por posição
def calcular_pontuacao_total_e_media(df_jogadores):
    if df_jogadores.empty:
        print("⚠️ Sem dados de jogadores para cálculo de pontuação.")
        return pd.DataFrame()

    df_total_pontuacao = df_jogadores.groupby('Posição', observed=True).agg(
        Pontuacao_Total=('Pontos', 'sum'),
        Quantidade=('Quantidade', 'sum')
    ).reset_index()

    df_total_pontuacao['Pontuacao_Media'] = df_total_pontuacao['Pontuacao_Total'] / df_total_pontuacao['Quantidade']

    # Definir ordem de exibição
    ordem_posicoes = ['Goleiro', 'Lateral', 'Zagueiro', 'Meia', 'Atacante', 'Técnico']
    df_total_pontuacao['Posição'] = pd.Categorical(df_total_pontuacao['Posição'], categories=ordem_posicoes, ordered=True)
    df_total_pontuacao = df_total_pontuacao.sort_values(by='Posição')

    return df_total_pontuacao


In [73]:
nome_time = "OlhaEleAiF.C!"
top_n = 4  # Altere se quiser

jogadores_mais_escalados_por_posicao = listar_jogadores_mais_escalados_por_posicao(df_escalacoes, nome_time, top_n)

if not jogadores_mais_escalados_por_posicao.empty:
    print(f"📋 Jogadores mais escalados por posição para o time {nome_time}:")
    display(jogadores_mais_escalados_por_posicao)

    pontuacao_total_e_media = calcular_pontuacao_total_e_media(jogadores_mais_escalados_por_posicao)

    print("📊 Pontuação total e média por posição:")
    display(pontuacao_total_e_media)

📋 Jogadores mais escalados por posição para o time OlhaEleAiF.C!:


,Posição,Nome,Clube,Pontos,Quantidade
14,Goleiro,Fábio,FLU,3.20,2
11,Goleiro,Caíque França,SPT,8.10,1
12,Goleiro,Cássio,CRU,10.20,1
13,Goleiro,Everson,CAM,0.60,1
17,Lateral,Bernabéi,INT,27.50,3
21,Lateral,Samuel Xavier,FLU,0.40,2
16,Lateral,Andrés Hurtado,RBB,4.60,1
18,Lateral,Ewerthon,JUV,1.20,1
43,Zagueiro,Gustavo Henrique,COR,10.80,3
42,Zagueiro,Félix Torres,COR,4.60,2


📊 Pontuação total e média por posição:


,Posição,Pontuacao_Total,Quantidade,Pontuacao_Media
0,Goleiro,22.10,5,4.42
1,Lateral,33.70,7,4.81
2,Zagueiro,36.70,8,4.59
3,Meia,82.30,10,8.23
4,Atacante,89.80,10,8.98
5,Técnico,27.29,5,5.46
